In [9]:
# Load package for anti-join to remove dbSNP sites
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
dbsnp_file <- read.table('/mnt/vast/hpc/csg/hcs2152/ZFR_RNA_Editing/References/Danio_rerio_dbSNP.txt',header=TRUE,sep='\t')
editing_sites <- read.table('/mnt/vast/hpc/csg/hcs2152/ZFR_RNA_Editing/JACUSA2/all_dpf/JACUSA2_all_dpf_sites.txt',header=TRUE,sep='\t')

In [5]:
# Add in JACUSA2 column names since they were removed (refer to file for exact column names)
colnames(editing_sites) <- c("contig","start","end","name","score","strand","bases11", "bases12","bases13","bases14","bases15",	"bases16", "bases21","bases22","bases23","bases24","bases25","bases26","info","filter","ref")

In [8]:
#Remove end column because start column is the site of interest (checked via GenomeBrowser)
editing_sites <- editing_sites[, !colnames(editing_sites) %in% c("end")]

In [12]:
#Put dbsnp and editing file into ID format --> contig_coordinate
dbsnp_file$ID <- paste(dbsnp_file$genomic.region, dbsnp_file$coordinate, sep = '_')

# Move 'ID' column to the beginning
dbsnp_file <- dbsnp_file[, c('ID', names(dbsnp_file)[-which(names(dbsnp_file) %in% 'ID')])]

#Put sites file into same ID format
editing_sites$ID <- paste(editing_sites$contig, editing_sites$start, sep = '_')

# Move 'ID' column to the beginning
editing_sites <- editing_sites[, c('ID', names(editing_sites)[-which(names(editing_sites) %in% 'ID')])]


In [13]:
#Note the number of editing sites before (number of rows)
dim(editing_sites)

[1] 11641839       21

In [14]:
# Anti-join to remove matching rows from editing_sites
editing_sites <- anti_join(editing_sites, dbsnp_file, by = 'ID')

In [15]:
# Check to see if sites were removed
dim(editing_sites)

[1] 11546071       21

In [16]:
output <- '/mnt/vast/hpc/csg/hcs2152/ZFR_RNA_Editing/JACUSA2/all_dpf/JACUSA2_all_dpf_sites_no_SNPS.txt'

In [17]:
write.table(editing_sites, file = output, sep = "\t", quote = FALSE, row.names = FALSE)